In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import  preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt
from mochi import *


In [38]:
from bs4 import BeautifulSoup
import string

In [2]:
#try xgboost
#fucntion from SRK
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
     seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1,\
     max_depth = 6,cv_dict = None,verbose_eval=True):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = eta
    param['max_depth'] = max_depth
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y,feature_names=feature_names)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y,feature_names=feature_names)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,\
        early_stopping_rounds=early_stop,evals_result = cv_dict,verbose_eval = verbose_eval)
    else:
        xgtest = xgb.DMatrix(test_X,feature_names=feature_names)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model


In [3]:
#lodaing data
data_path = "../../kaggleData/2sigma/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)


(49352, 15)
(74659, 14)


In [4]:
#basic numerical features
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]

In [5]:
#some transfromed features
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour

#some new numerical features related to the price
train_df["price_per_bath"] =  (train_df["price"]*1.0/train_df["bathrooms"]).replace(np.Inf,-1)
train_df["price_per_bed"] = (train_df["price"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["bath_per_bed"] = (train_df["bathrooms"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["price_per_room"] = (train_df["price"]*1.0/(train_df["bedrooms"]+train_df["bathrooms"])).replace(np.Inf,-1)

# adding all these new features to use list # "listing_id",
features_to_use.extend(["num_photos", "num_features", "num_description_words",\
                        "created_year","listing_id", "created_month", "created_day", "created_hour"])
#price new features
features_to_use.extend(["price_per_bed","bath_per_bed","price_per_room"])


In [6]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 42)

In [25]:
def stripTagsAndUris(x):
    uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>???“”‘’]))'
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""

In [64]:
#define punctutaion filter
def removePunctuationAndNumber(x):
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    removing = string.punctuation
    removed = re.sub("["+removing+"]", " ", x)
    removed = re.sub(r"\d",'',removed)
    return removed

In [80]:
def getUpperTokens(tokenList):
    number = 0
    for word in tokenList:
        if re.match('[A-Z]{2,}',word)!=None:
            number+=1
    if len(tokenList) > 0:
        return number*1.0/len(tokenList)
    else:
        return -1

In [81]:
temp = train_df['description']\
       .apply(stripTagsAndUris)\
       .apply(removePunctuationAndNumber)\
       .apply(lambda x : x.split() if x is not None else [])\
       .apply(getUpperTokens)

In [7]:
#running and getting the cv from xgboost
cv_scores = []
cv_result = []

#K-FOLD already defined.If not ,use
#KF=KFold(len(train_X),5,shuffle=True,random_state = 42)
i=0
for dev_index, val_index in KF:
        result_dict = {}
        
        """some preprocessing like feature constructed in cv manners"""
        
        #split the orginal train set into dev_set and val_set
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        
        dev_X, val_X = dev_set[features_to_use].as_matrix(), val_set[features_to_use].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        
        """ 
         runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
         seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1,\
         max_depth = 6,cv_dict = None):
         """
        
        preds, model = runXGB(dev_X, dev_y, val_X, val_y,early_stop  = 20,\
                              feature_names = features_to_use,cv_dict = result_dict,verbose_eval=True)
        loss = log_loss(val_y, preds)
        cv_scores.append(loss)
        cv_result.append(result_dict)
        i+=1
        print 'loss for the turn '+str(i)+' is '+str(loss)

[0]	train-mlogloss:1.04113	test-mlogloss:1.04253
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:0.990764	test-mlogloss:0.993263
[2]	train-mlogloss:0.951165	test-mlogloss:0.954688
[3]	train-mlogloss:0.913401	test-mlogloss:0.918126
[4]	train-mlogloss:0.880869	test-mlogloss:0.886504
[5]	train-mlogloss:0.852162	test-mlogloss:0.858837
[6]	train-mlogloss:0.826243	test-mlogloss:0.833794
[7]	train-mlogloss:0.803247	test-mlogloss:0.811797
[8]	train-mlogloss:0.784647	test-mlogloss:0.794119
[9]	train-mlogloss:0.767616	test-mlogloss:0.778038
[10]	train-mlogloss:0.752352	test-mlogloss:0.763476
[11]	train-mlogloss:0.738303	test-mlogloss:0.750363
[12]	train-mlogloss:0.725814	test-mlogloss:0.738471
[13]	train-mlogloss:0.714862	test-mlogloss:0.728451
[14]	train-mlogloss:0.704626	test-mlogloss:0.719134
[15]	train-mlogloss:0.695417	test-mlogloss:0.710734
[16]	train-mlogloss:0.686302	

KeyboardInterrupt: 

In [12]:
#plot the validation curv
cvResult = CVstatistics(cv_result,'mlogloss')
cvResult.turns

test0    254
test1    260
test2    342
test3    296
test4    273
Name: 341, dtype: int32

In [14]:
#cvResult.validCurve(stop=False)
#some errors at certain turn to see the descending
cv_scores
np.mean(cv_scores)

0.58505953557896273

In [15]:
#show the importance of the features
showImportance(model,'gain')

[('bathrooms', 0.09725565800916634), ('price', 0.09499099365937312), ('price_per_room', 0.09205531264056366), ('created_hour', 0.09194288123746672), ('price_per_bed', 0.08666569315447636), ('num_photos', 0.07428368001559033), ('bedrooms', 0.07012971325635668), ('bath_per_bed', 0.061415666612366956), ('num_features', 0.06043546567125035), ('latitude', 0.058875420604080596), ('longitude', 0.05741437403240373), ('num_description_words', 0.04480527126354566), ('listing_id', 0.042255845208890516), ('created_day', 0.03503384908900436), ('created_month', 0.03244017554546452)]
